In [114]:
!pip install gymnasium

In [115]:
import gymnasium as gym
# استيراد مكتبة الحالات
from gymnasium import spaces
# مكتبة نمباي للتعامل مع المصفوفات
import numpy as np
# مكتبة العشوائية
import random

In [116]:
# مكتبة الرسم
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [117]:
# الصنف المخصص
class DroneEnvironment(gym.Env):

    # توليد رقم الحالة من موضع معين
    def position_to_state(self, position):
      # إحداثيات الموضع
      x1, y1, z1 = position
      # إحداثيات الهدف
      x2, y2, z2 = self.goal_position

      # تحويل الموضع إلى رقم
      state1 = x1 * self.grid_size**2 + y1 * self.grid_size + z1
      # تحويل الهدف إلى رقم
      state2 = x2 * self.grid_size**2 + y2 * self.grid_size + z2

      # التحويل لرقم حالة فريد
      state = state1 * (self.grid_size**3) + state2
      return state

    # إيجاد الموضع من رقم الحالة
    def state_to_position(self, state):
      # إيجاد الرقم الموافق للموضع من رقم الحالة
      state1 = state // (self.grid_size**3)
      # إيجاد إحداثيات الموضع
      x1 = state1 // (self.grid_size**2)
      y1 = (state1 % (self.grid_size**2)) // self.grid_size
      z1 = state1 % self.grid_size
      return [x1, y1, z1]


    # باني الصنف
    def __init__(self, grid_size=10, start_position=None, buildings=None, prohibited_zones=None,max_episode_steps=200):
        self.grid_size = grid_size
        self.start_position = start_position
        self.goal_position = [0,0,0]
        self.buildings = buildings if buildings else []
        self.prohibited_zones = prohibited_zones if prohibited_zones else []
        # عدد الخطوات الأعظمي في الحلقة الواحدة
        self.max_episode_steps=max_episode_steps
        # عدد الخطوات التي تم القيام بها
        self.steps_taken=0

        # مسار الدرون
        self.path=[]
        # بداية المسار من نقطة الانطلاق
        self.path.append(start_position)

        # الحالة الحالية
        self.current_state =self.position_to_state(self.start_position)

        # الأفعال
        self.actions = [(1, 0, 0), (-1, 0, 0), (0, 1, 0), (0, -1, 0), (0, 0, 1), (0, 0, -1)]

        # فضاء الأفعال
        self.action_space = spaces.Discrete(len(self.actions))

        # فضاء الحالات
        self.observation_space = spaces.Discrete(self.grid_size**6)

        # الرسم
        self.fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

     # اختبار الاصطدام مع بناء
    def is_in_building(self, position, building):
        # إحداثيات زاوية البناء اليسرى السفلى
        bx, by, bz = building['pos']
        # عرض وطول وارتفاع
        bsx, bsy, bsz = building['size']
        # اختيار الاصطدام
        if bx <= position[0] <= bx + bsx and by <= position[1] <= by + bsy and bz <= position[2] <= bz + bsz:
            return True
        return False

    # اختبار الوقوع ضمن منطقة ممنوعة
    def is_in_prohibited_zone(self, position, zone):
        zx, zy, zz = zone['pos']
        zsx, zsy, zsz = zone['size']
        if zx <= position[0] <= zx + zsx and zy <= position[1] <= zy + zsy:
            return True
        return False

   #  الحركة
    def step(self, action):
        # زيادرة عدد حركات الحلقة
        self.steps_taken+=1
        # اختبار تجاوز الحد الأعظمي
        if self.steps_taken>self.max_episode_steps:
            return self.current_state, -100, True, False, {}

        # الموقع
        x,y,z=self.state_to_position(self.current_state)

        # الموقع التالي
        next_position = [
            x + self.actions[action][0],
            y + self.actions[action][1],
            z + self.actions[action][2]]

        # اختبار الخروج من الشبكة
        if any(coord < 0 or coord >= self.grid_size for coord in next_position):
            return self.current_state, -100, False, False, {}
        # اختبار الاصطدام مع بناء
        for building in self.buildings:
            if self.is_in_building(next_position, building):
                return self.current_state, -50, False, False, {}
        # اختبار الوقوع في منطقة ممنوعة
        for zone in self.prohibited_zones:
            if self.is_in_prohibited_zone(next_position, zone):
                return self.current_state, -50, False, False, {}

        # اختبار الوصول للهدف
        if next_position == self.goal_position:
            self.path.append(next_position)
            return self.position_to_state(next_position), 1000,  False, True, {}

        # الانتقال للحالة التالية
        self.current_state = self.position_to_state(next_position)

        # الإضافة لمسار الدرون
        self.path.append(next_position)

        return self.current_state, -1, False, False, {}

    # إعادة التهيئة
    def reset(self, seed=None, return_info=False, options=None):
        # مسح الرسم السابق
        self.fig.data = []
        # مسح المسار
        self.path=[]
        # إضافة موضع البداية إلى المسار
        self.path.append(self.start_position)
        # الحالة الابتدائية
        self.current_state = self.position_to_state(self.start_position)
         # عدد خطوات الدرون
        self.steps_taken=0

        # توليد موضع هدف عشوائي
        while True:
          accept_position=True
          x = random.randint(0, self.grid_size-1)
          y = random.randint(0, self.grid_size-1)
          z = random.randint(0, self.grid_size-1)
          position=[x,y,z]
          # اختبار الاصطدام مع بناء
          for building in self.buildings:
              if self.is_in_building(position, building):
                  accept_position=False
                  break
          # اختبار الوقوع في منطقة ممنوعة
          for zone in self.prohibited_zones:
              if self.is_in_prohibited_zone(position, zone):
                accept_position=False
                break
          if accept_position:
            break
        self.goal_position=position

        return self.current_state,{}

    # تحديد موضع هدف معين
    def set_goal(self, goal):
      self.goal_position=goal

    # رسم البداية والهدف
    def plot_start_goal(self):
        self.fig.add_trace(go.Scatter3d(
            x=[self.start_position[0]], y=[self.start_position[1]], z=[self.start_position[2]],
            mode='markers', marker=dict(size=10, color='green'), name='Start', showlegend=False
        ))
        self.fig.add_trace(go.Scatter3d(
            x=[self.goal_position[0]], y=[self.goal_position[1]], z=[self.goal_position[2]],
            mode='markers', marker=dict(size=10, color='blue'), name='Goal', showlegend=False
        ))

    # رسم مسار الدرون
    def plot_path(self):
        x_vals = [pos[0] for pos in self.path]
        y_vals = [pos[1] for pos in self.path]
        z_vals = [pos[2] for pos in self.path]
        self.fig.add_trace(go.Scatter3d(
            x=x_vals, y=y_vals, z=z_vals,
            mode='lines+markers', line=dict(color='black', width=2), marker=dict(size=2, color='red'),name='Drone Path', showlegend=False))

    # رسم متوازي المستطيلات
    def plot_poly(self, position, size, color):
        x, y, z = position
        lx, ly, lz = size
        # الزوايا الثمانية
        corners = np.array([[x, y, z],
                            [x + lx, y, z],
                            [x + lx, y + ly, z],
                            [x, y + ly, z],
                            [x, y, z + lz],
                            [x + lx, y, z + lz],
                            [x + lx, y + ly, z + lz],
                            [x, y + ly, z + lz]])

        # الوحوه السته
        vertices = [[corners[0], corners[1], corners[2], corners[3]],
                    [corners[4], corners[5], corners[6], corners[7]],
                    [corners[0], corners[1], corners[5], corners[4]],
                    [corners[1], corners[2], corners[6], corners[5]],
                    [corners[2], corners[3], corners[7], corners[6]],
                    [corners[3], corners[0], corners[4], corners[7]]]

        # الرسم
        for v in vertices:
            self.fig.add_trace(go.Scatter3d(
                x=[p[0] for p in v], y=[p[1] for p in v], z=[p[2] for p in v],
                mode='lines', line=dict(color=color, width=4), name='Building', showlegend=False
            ))


    # دالة الإظهار
    def render(self):
      # مسح الرسم السابق
      self.fig.data = []
      # تحديث الرسم
      self.fig.update_layout(
          scene=dict(
              xaxis=dict(range=[0, self.grid_size-1], title='X'),
              yaxis=dict(range=[0, self.grid_size-1], title='Y'),
              zaxis=dict(range=[0, self.grid_size-1], title='Z')
          ),
          title="Drone Environment"
      )
      # رسم الأبنية
      for building in self.buildings:
          self.plot_poly(building['pos'], building['size'], color="yellow")

      # رسم المناطق الممنوعة
      for zone in self.prohibited_zones:
          self.plot_poly(zone['pos'], zone['size'], color="red")

      # رسم البداية والنهاية
      self.plot_start_goal()

      # رسم المسار
      self.plot_path()

      # رسم الموضع الحالي
      if self.current_state is not None:
          x,y,z= self.state_to_position(self.current_state)
          self.fig.add_trace(go.Scatter3d(
              x=[x], y=[y], z=[z],
              mode='markers', marker=dict(size=5, color='red'), name='Drone', showlegend=False
          ))

      # إظهار الرسم
      self.fig.show()


    # الإغلاق
    def close(self):
        self.fig.data = []


In [118]:
# تسجيل البيئة
gym.register(
    id="Drone-v0",
    entry_point=DroneEnvironment
)

In [119]:
# مثال
# الأبنية
buildings = [
    {'pos': [3, 3, 0], 'size': [5, 5, 5]},
    {'pos': [8, 8, 0], 'size': [1, 1, 1]}
]
# المناطق الممنوعة
prohibited_zones = [
    {'pos': [0, 2, 0], 'size': [2, 4, 4]},
    {'pos': [8, 1, 0], 'size': [2, 2, 5]}
]
# البداية
start_position = [1, 9, 0]
# الهدف
goal_position = [9, 0, 8]

# إنشاء البيئة
env = gym.make("Drone-v0", grid_size=10, start_position=start_position,buildings=buildings, prohibited_zones=prohibited_zones , max_episode_steps=40)

env.reset()
raw_env = env.unwrapped
raw_env.set_goal(goal_position )



# الإظهار
env.render()

In [120]:
# مكتبة الزمن
import time
# مكتبة الإظهار على كولاب
from IPython.display import clear_output

In [121]:
# مكتبة الزمن
import time

terminated=False # الوصول للهدف
truncated=False # انتهاء دورات الحلقة
# مسح كل الإظهار
clear_output()
# إعادة التهيئة
env.reset()
# تحديد هدف معين
raw_env = env.unwrapped
raw_env.set_goal(goal_position)
# إجراء حلقة للتفاعل مع البيئة
while not (terminated or truncated):
     # اختيار فعل عشوائي
    action = env.action_space.sample()
     # تنفيذ الفعل في البيئة
    state, reward, terminated, truncated, info  = env.step(action)
    # عرض البيئة بعد كل خطوة
    env.render()
    # التوقف لثانية
    time.sleep(1)
    # مسح الإظهار
    #clear_output()

In [122]:
# مسح الإظهار
clear_output()
# الإظهار
env.render()


In [123]:
# إغلاق البيئة
env.close()